In [2]:
import os
import torch
from torch.utils.data import DataLoader
from models.pointmlp import pointMLP 
from intra import IntrA
from Augment.pointaugment import PointAugment
from Augment.augmentor import Augmentor

dev = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

ModuleNotFoundError: No module named 'pointnet2_ops'

In [ ]:
trn = IntrA(
    "/data",
    npoints=1024,
    exclude_seg=True,
    norm=False,
    dataset="classification",
    fold=1,
    kfold_splits="./file_splits/cls",
    test=False,
)

tst = IntrA(
    "/data",
    npoints=1024,
    exclude_seg=True,
    norm=False,
    dataset="classification",
    fold=1,
    kfold_splits="./file_splits/cls",
    test=True,
)

train_dl = DataLoader(trn, batch_size=16, num_workers=4, drop_last=True, shuffle=True)
test_dl = DataLoader(tst, batch_size=8, num_workers=2, drop_last=False)

classifier = pointMLP(num_classes=2)
augmentor = Augmentor()

optimizer_c = torch.optim.SGD(
    classifier.parameters(),
    lr=0.025,
    momentum=0.9,
    weight_decay=2e-4,
)

optimizer_a = torch.optim.Adam(
    augmentor.parameters(),
    lr=0.001,
    betas=(0.9, 0.999),
    eps=1e-08,
    weight_decay=1e-4,
)

epochs = 3

scheduler_c = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer_c, epochs, eta_min=5e-4, last_epoch=-1
)

# scheduler_a = torch.optim.lr_scheduler.StepLR(
#     optimizer_a,
#     step_size=20,
#     gamma=0.5
# )
scheduler_a = None

adverse = PointAugment(
    classifier,
    augmentor,
    train_dl,
    test_dl,
    epochs,  
    optimizer_a,
    optimizer_c,
    scheduler_a,
    scheduler_c,
)
